# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [1]:
# Your code here; import the necessary packages
import pandas as pd
import sqlite3

conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [2]:
qry1 = """
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
"""
pd.read_sql(qry1,conn)

customerNumber contactLastName contactFirstName
0             141          Freyre           Diego

In [3]:
# Your code here
qry = """
SELECT 
    customerNumber,
    contactLastname,
    contactFirstname
FROM customers
WHERE customerNumber IN (SELECT customerNumber
                         FROM orders
                         WHERE orderDate = '2003-01-31')
;
"""
pd.read_sql(qry,conn)

customerNumber contactLastName contactFirstName
0             141          Freyre           Diego

## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [7]:
# Your code here
orders_per_product = """
SELECT 
    productName,
    MSRP,
    COUNT(orderNumber) AS net_orders,
    SUM(quantityOrdered) AS gross_units
FROM products
JOIN orderdetails 
    USING (productCode)
GROUP BY productName
ORDER BY gross_units DESC
;
"""
pd.read_sql(orders_per_product,conn)

productName    MSRP  net_orders  gross_units
0                1992 Ferrari 360 Spider red  169.34          53         1808
1                       1937 Lincoln Berline  102.74          28         1111
2                  American Airlines: MD-11S   74.03          28         1085
3    1941 Chevrolet Special Deluxe Cabriolet  105.87          28         1076
4               1930 Buick Marquette Phaeton   43.64          28         1074
..                                       ...     ...         ...          ...
104             1999 Indy 500 Monte Carlo SS  132.00          25          855
105                       1911 Ford Town Car   60.54          25          832
106         1936 Mercedes Benz 500k Roadster   41.03          25          824
107               1970 Chevy Chevelle SS 454   73.49          25          803
108                    1957 Ford Thunderbird   71.27          24          767

[109 rows x 4 columns]

## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [10]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
product_by_buyers = """
SELECT 
    productName,
    COUNT(DISTINCT customerNumber) AS unique_customers
FROM products 
JOIN orderdetails
    USING(productCode)
JOIN orders
    USING(orderNumber)
GROUP BY productName
ORDER BY unique_customers DESC
;
"""
pd.read_sql(product_by_buyers,conn)

productName  unique_customers
0            1992 Ferrari 360 Spider red                40
1                       Boeing X-32A JSF                27
2                    1972 Alfa Romeo GTA                27
3               1952 Alpine Renault 1300                27
4                     1934 Ford V8 Coupe                27
..                                   ...               ...
104  1958 Chevy Corvette Limited Edition                19
105                  2002 Chevy Corvette                18
106            1969 Chevrolet Camaro Z28                18
107                    1952 Citroen-15CV                18
108                   1949 Jaguar XK 120                18

[109 rows x 2 columns]

## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [12]:
# Your code here
complicated = """
SELECT 
    DISTINCT employeeNumber,
    employees.firstName,
    employees.lastName,
    offices.city,
    officeCode
FROM employees 
JOIN offices
    USING(officeCode)
JOIN customers 
    ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
WHERE productCode IN (
                      SELECT productCode
                      FROM products
                      JOIN orderdetails
                          USING(productCode)
                      JOIN orders
                          USING(orderNumber)
                      GROUP BY productCode
                      HAVING COUNT(DISTINCT customerNumber) < 20
                      )
;
"""
pd.read_sql(complicated,conn)

employeeNumber firstName   lastName           city  officeCode
0             1370    Gerard  Hernandez          Paris           4
1             1501     Larry       Bott         London           7
2             1337      Loui     Bondur          Paris           4
3             1166    Leslie   Thompson  San Francisco           1
4             1286  Foon Yue      Tseng            NYC           3
5             1612     Peter      Marsh         Sydney           6
6             1611      Andy     Fixter         Sydney           6
7             1401    Pamela   Castillo          Paris           4
8             1621      Mami      Nishi          Tokyo           5
9             1323    George     Vanauf            NYC           3
10            1165    Leslie   Jennings  San Francisco           1
11            1702    Martin     Gerard          Paris           4
12            1216     Steve  Patterson         Boston           2
13            1188     Julie   Firrelli         Boston           2
14            1504     Barry      Jones         London           7

## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [13]:
# Your code here
less_complicated = """
SELECT
    employeeNumber,
    firstName,
    lastName,
    COUNT(customerNumber) AS num_of_customers
FROM employees
JOIN customers
    ON employees.employeeNumber = customers.salesRepEmployeeNumber
GROUP BY employeeNumber
HAVING AVG(creditLimit) > 15000
;
"""
pd.read_sql(less_complicated,conn)

employeeNumber firstName   lastName  num_of_customers
0             1165    Leslie   Jennings                 6
1             1166    Leslie   Thompson                 6
2             1188     Julie   Firrelli                 6
3             1216     Steve  Patterson                 6
4             1286  Foon Yue      Tseng                 7
5             1323    George     Vanauf                 8
6             1337      Loui     Bondur                 6
7             1370    Gerard  Hernandez                 7
8             1401    Pamela   Castillo                10
9             1501     Larry       Bott                 8
10            1504     Barry      Jones                 9
11            1611      Andy     Fixter                 5
12            1612     Peter      Marsh                 5
13            1621      Mami      Nishi                 5
14            1702    Martin     Gerard                 6

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!